<a href="https://colab.research.google.com/github/laienwu/WCS/blob/master/project3_recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
url='/content/gdrive/My Drive/Colab Notebooks/Project3/movie_metadata.csv'
data=pd.read_csv(url)

In [ ]:
df1=df.cpoy()

In [ ]:
data_use=data.loc[:,['genres','plot_keywords','movie_title','actor_1_name','actor_2_name','actor_3_name','director_name','imdb_score']]
data_use['movie_title']=[i.replace("\xa0","") for i in list(data_use['movie_title'])]

In [ ]:
print(data_use.shape)
clean_data3 = data_use.dropna(axis = 0)
print(clean_data3.shape)
clean_data3 = clean_data3.drop_duplicates(['movie_title'])
clean_data3 = clean_data3.reset_index(drop = True)
print(clean_data3.shape)
clean_data3.head()

(5043, 8)
(4781, 8)
(4659, 8)


,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score
0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,Avatar,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,7.9
1,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,Gore Verbinski,7.1
2,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Sam Mendes,6.8
3,Action|Thriller,deception|imprisonment|lawlessness|police offi...,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Christopher Nolan,8.5
4,Action|Adventure|Sci-Fi,alien|american civil war|male nipple|mars|prin...,John Carter,Daryl Sabara,Samantha Morton,Polly Walker,Andrew Stanton,6.6


In [ ]:
people_list = []
for i in range(clean_data3.shape[0]):
    name1 = clean_data3.loc[i, 'actor_1_name'].replace("_", " ")
    name2 = clean_data3.loc[i, 'actor_2_name'].replace("_", " ")
    name3 = clean_data3.loc[i, 'actor_3_name'].replace("_", " ")
    name4 = clean_data3.loc[i, 'director_name'].replace("_", " ")
    people_list.append("\xa0".join([name1, name2, name3, name4]))
clean_data3['people'] = people_list

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def token(text):
    return(text.split("_"))

cv_kw=CountVectorizer(max_features=100,tokenizer=token )
keywords = cv_kw.fit_transform(clean_data3['plot_keywords'])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data3['genres'])
genres_list = ['genres_' + i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data3['plot_keywords'])
people_list = ["pp_" + i for i in cv_pp.get_feature_names()]

cluster_data=np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list=keywords_list+genres_list+people_list

In [ ]:
from sklearn.cluster import KMeans
mod = KMeans(n_clusters=100)
category = mod.fit_predict(cluster_data)
category_dataframe = pd.DataFrame({"category":category}, index = clean_data3['movie_title'])

In [ ]:
#clean_data3.loc[list(category_dataframe['category'] == 0), ['genres', 'movie_title', 'people']]

In [ ]:
def recommend (movie_name,recommend_number= 5) : 
  if movie_name in list(clean_data3['movie_title']):
    movie_cluster = category_dataframe.loc[movie_name, 'category']
    print("movie_cluster :", movie_cluster)
    score = clean_data3.loc[list(category_dataframe['category'] == movie_cluster),
                          ['imdb_score', 'movie_title']]
    sort_score = score.sort_values(['imdb_score'],ascending=False)
    sort_score = sort_score[sort_score['movie_title'] != movie_name]
    recommend_number = min(sort_score.shape[0], recommend_number)
    print(recommend_number)
    recommend_movie = list(sort_score.iloc[range(recommend_number),1])
    print(recommend_movie)
  else:
    print("Can't find this movie!")

recommend('Nikita')    

Can't find this movie!


In [ ]:
clean_data3[clean_data3['movie_title'].values=='Daredevil']

,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score,people


In [ ]:
recommend('The Host')

movie_cluster : 0
5
['The Good, the Bad and the Ugly', 'Queen of the Mountains', 'Seven Samurai', 'The Silence of the Lambs', 'Spirited Away']


In [ ]:
clean_data3[clean_data3['movie_title'] == 'Avatar']

,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score,people
0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,Avatar,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,7.9,CCH Pounder Joel David Moore Wes Studi James C...


In [ ]:
clean_data3[clean_data3['movie_title'] == 'Star Wars: Episode V - The Empire Strikes Back']

,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score,people
1983,Action|Adventure|Fantasy|Sci-Fi,duel|famous twist|rebel|rescue|snowy landscape,Star Wars: Episode V - The Empire Strikes Back,Harrison Ford,Kenny Baker,Anthony Daniels,Irvin Kershner,8.8,Harrison Ford Kenny Baker Anthony Daniels Irvi...


In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
pip install wordcloud